In [ ]:
import numpy as np
import pandas as pd
import sklearn.metrics as skm
from sklearn.preprocessing import MinMaxScaler
from time import time

In [ ]:
t1 = pd.read_csv('cosine.csv')

In [ ]:
# scaler = MinMaxScaler()
# scaler.fit(t1)

In [ ]:
# print(scaler.transform(t1))

In [ ]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [ ]:
test['rating'] = test['rating'].round().astype(int)
test['uId'] = test['uId'].round().astype(int)
test['iId'] = test['iId'].round().astype(int)
train['rating'] = train['rating'].round().astype(int)
train = train.to_numpy()
test = test.to_numpy()


In [ ]:
matrix = np.zeros((943,1683),dtype=int)
for i in train:
        matrix[i[1]-1,i[0]-1] = i[2]

In [ ]:
def PredRating(user,item,similarity):
    try:
        top = similarity[user].argsort()[1:100]
    except IndexError:
        sum,count = 0,0
        for j in range(1682):
            if matrix[user,j] != 0:
                count+=1
                sum+=1
        return sum/count
    temp,avgUh,simi = [],[],[]
    for i in top:
        if matrix[i,item] !=0:
            temp.append(i)
            simi.append(similarity[user,i])
    temp.append(user)
    for i in temp:
        sum,count = 0,0
        for j in range(1682):
            if matrix[i,j] != 0:
                count+=1
                sum+=matrix[i,j]
        avgUh.append(sum/count)
    avgU = np.nan_to_num(np.array(avgUh.pop()))
    temp.pop()
    if len(temp)==0:
        return avgU
    simi = np.nan_to_num(np.array(simi))
    num = (simi*(avgUh-avgU)).sum()
    den = simi.sum()
    # for i in range(len(temp)):
    #     num+=similarity[user,temp[i]]*avgUh[i]
    #     den+=similarity[user,temp[i]]
    try:
        result = int(round((avgU+num/den),0))
    except ZeroDivisionError:
        result = 0
    return result



In [ ]:
cosine = pd.read_csv('cosine.csv').to_numpy()
cosinePred = np.zeros(len(test),dtype=int)

In [ ]:
# %%time
j = 0
for i in test:
    cosinePred[j] = PredRating(i[1]-1,i[0]-1,cosine)
    j+=1
    # if j==100:
    #   break
test = pd.read_csv('test.csv')
cosinePred = pd.DataFrame({'cosine':cosinePred})
frames = [test,cosinePred]
t = pd.concat(frames,axis=1)
t.to_csv('cosinePred.csv',sep=',',index=False)
# from google.colab import files
# files.download('cosinePred.csv')